In [1]:
#Import our dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#Set a path to chrome and setup our url
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\Origin\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [23]:
#Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
#set a slight delay in page loading
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

The final line of code in above cell searches tag: div and with class: item_list  

In [24]:
#Set up HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text') #The parent element is stored in slide_elem

In [5]:
#Obtain the title of the first article:
slide_elem.find('div', class_='content_title')

<div class="content_title">My Culture, My Voice</div>

In [6]:
#Filter to get just the text of the title:
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'My Culture, My Voice'

In [7]:
#Get the summary of the first article
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'In honor of Hispanic Heritage Month, Christina Hernandez, an instrument engineer on the Mars 2020 mission, talks about her childhood and journey to NASA.'

### Featured images

In [8]:
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [9]:
#find and click the full image button
full_image_elem = browser.find_by_tag('button')[1] #First button element in webpage html
full_image_elem.click()

In [10]:
html = browser.html
img_soup = soup(html, 'html.parser')

In [11]:
#Find the relative (featured) image url:
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel
#Note that get('src') pulls a link to that image

'image/featured/mars1.jpg'

In [12]:
#Now we need to add this to the base url to get a complete url:
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url



'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [13]:
#We are now going to scrape the mars facts table from https://galaxyfacts-mars.com/
#read_html will automatically print all tables found in a particular html
df = pd.read_html('https://galaxyfacts-mars.com/')[0]  #The zero tell pandas to import first table it sees
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace = True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [14]:
#We can convert the dataframe back to html format:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [20]:
#End our session (very important as it shuts down the automated browser)
browser.quit()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

## Hemispheres

In [239]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [240]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
mars_hemi = soup(html, 'html.parser')
div_descriptions = mars_hemi.find_all('div', class_= 'description')
def get_hemisphere_data(description):
    title = description.find('h3').get_text()
    hemi_link = description.find('a', class_= 'itemLink product-item').get('href')
    hemi_link_url = f'{url}{hemi_link}'
    browser.visit(hemi_link_url)
    html = browser.html
    mars_img = soup(html, 'html.parser')
    div_download = mars_img.find('div', class_='downloads')
    image_href = div_download.find('a').get('href')
    link = f'{url}{image_href}'
    return title, link
   

In [241]:
for div_description in div_descriptions:
    dic = {}
    get_hemisphere_data(div_description)
    dic['title'] = get_hemisphere_data(div_description)[0]
    dic['image_url'] = get_hemisphere_data(div_description)[1]
    hemisphere_image_urls.append(dic)

In [242]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]

In [243]:
# 5. Quit the browser
browser.quit()